In [40]:
import os
import pandas as pd
import json
import numpy as np
#!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
#from ipynb.fs.full.my_functions import factorial

In [42]:
import tweepy
import traceback

In [43]:
import spacy
import torch

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install spacymoji
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#!pip install transformers
from transformers import AutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

from sklearn.utils import resample
from sklearn.model_selection import GroupShuffleSplit

In [47]:
# Get features for TF-IDF
tfidf = TfidfVectorizer()#tokenizer=spacy_tokeniser)
tfidf.fit(train_text)
train_features = tfidf.transform(train_text)
#validation_features = tfidf.transform(val_text)
test_features = tfidf.transform(test_text)

In [54]:
#pickle.dump(tfidf, open("tfidf_vectorizer.sav", "wb"))
print(train_text)

204478    Responsibilities require you to be one hundred...
210630    #octinbooks18 Day Fourteen: Hogwarts Class You...
96763     Read the full transcript of Obama’s fiery anti...
258305    In 2004, a one-term senator from Illinois took...
95357     @steve_durnan @ThomasSowell Sure did and I not...
                                ...                        
108926    This would be so great if the DOJ/FBI would ar...
108927    Intel Officer Reveals Obama Could Be ‘First Pr...
108928    Intel Officer Reveals Obama Could Be ‘First Pr...
108929    BREAKING: Intel Officer Reveals Obama Could Be...
108930    Obama could be brought up on federal charges f...
Name: text, Length: 66259, dtype: object


In [48]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [38]:
shorter = train_features[:50, :50]
shorter_labels = train_labels[:50]
print(train_features[100, 666])
#print(train_features)
print(train_features.shape)
print(shorter.shape)

# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(shorter, shorter_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(shorter, shorter_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(shorter, shorter_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

0.0
(66259, 111786)
(50, 50)


C:\Users\karli\AppData\Local\Temp\ipykernel_4036\3339626443.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  shorter_labels = train_labels[:50]


ValueError: The number of classes has to be greater than one; got 1 class

In [49]:
# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(train_features, train_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(train_features, train_labels)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(train_features, train_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

Evaluation for: SVC test
              precision    recall  f1-score   support

       False      0.816     0.285     0.422     74981
        True      0.572     0.937     0.711     76622

    accuracy                          0.614    151603
   macro avg      0.694     0.611     0.566    151603
weighted avg      0.693     0.614     0.568    151603


Confusion matrix:
 [[21340 53641]
 [ 4824 71798]]
Evaluation for: LR (TF-IDF) test
              precision    recall  f1-score   support

       False      0.823     0.450     0.582     74981
        True      0.627     0.905     0.741     76622

    accuracy                          0.680    151603
   macro avg      0.725     0.678     0.662    151603
weighted avg      0.724     0.680     0.662    151603


Confusion matrix:
 [[33752 41229]
 [ 7256 69366]]
Evaluation for: Dummy MF test
              precision    recall  f1-score   support

       False      0.495     1.000     0.662     74981
        True      0.000     0.000     0.000    

# print(train_features)

In [50]:
# Save model
import pickle
pickle.dump(lr_model_tfidf, open("simple_model.sav", "wb"))

## Make Prediction

In [ ]:
# Get output

input_text = "Watch video showing gunfire inside Canada's pa"
input_df = pd.DataFrame({"src_text": [input_text]})
input_features = tfidf.transform(input_df)

predicted_label = lr_model_tfidf.predict(input_features)

print(predicted_label)

['0']


# Import and Fix Data

In [45]:
# Combine dataset classes
real_df = pd.read_csv("data/FakeNewsNet/dataset/pf_real.csv")
fake_df = pd.read_csv("data/FakeNewsNet/dataset/pf_fake.csv")
fake_df['true'] = False
real_df['true'] = True
df = pd.concat([fake_df, real_df])

# Fix imbalance
# https://elitedatascience.com/imbalanced-classes
# Separate majority and minority classes
df_majority = df[df.true==True]
df_minority = df[df.true==False]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=len(df_minority),     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

# Split train-test
# https://stackoverflow.com/questions/61337373/split-on-train-and-test-separating-by-group

# Update df to the downsampled one

X = df_downsampled.drop(labels='true', axis=1)
y = df_downsampled.true


# Split in training/testing data but in a way that Tweets referring to the same article don't appear in both datasets
gs = GroupShuffleSplit(n_splits=2, test_size=.6, random_state=0)
train_ix, test_ix = next(gs.split(X, y, groups=X.article_id))

print(type(train_ix))

X_train = X.iloc[train_ix]
y_train = y.iloc[train_ix]

X_test = X.iloc[test_ix]
y_test = y.iloc[test_ix]

<class 'numpy.ndarray'>


In [46]:
# Get features for TF-IDF

train_text = X_train['text']
train_labels = y_train

test_text = X_test['text']
test_labels = y_test

# BERT

In [7]:
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=URn-DWJt5xhP
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder

In [9]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [20]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [22]:
print(padded)

[[  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 [  101  4911  1024 ...     0     0     0]
 ...
 [  101 21907  2289 ...     0     0     0]
 [  101  4931  3071 ...     0     0     0]
 [  101  1030  3565 ...     0     0     0]]


In [13]:
padded_encoding = df['text'].apply(
    lambda src_text: tokenizer.encode_plus(
        src_text, add_special_tokens = True,
        truncation = True, padding = "max_length", 
        return_attention_mask = True, return_tensors = "pt")
)

In [14]:
#input_ids = torch.tensor(np.array(padded_encoding))

# print(padded_encoding.shape)

# #new_tensor = torch.tensor([200, padded_encoding])

new_tensor = padded_encoding[None, :]
print(new_tensor)
new_tensor[0] = 200
print(new_tensor.shape)

with torch.no_grad():
    last_hidden_states = model(new_tensor) # ERROR: Tries to get 255 GB of RAM

C:\Users\karli\AppData\Local\Temp\ipykernel_4036\3607074656.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  new_tensor = padded_encoding[None, :]


[[{'input_ids': tensor([[  101,  4911,  1024,  2034,  5088,  2136, 18806, 10528,  2058, 16916,
           24106,  1024, 16770,  1024,  1013,  1013,  1056,  1012,  2522,  1013,
            2852,  2475,  3501, 13687,  2243,  2078,  2581,  2278,   102,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
               0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
              

(1, 405102)


TypeError: 'int' object is not callable

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.5866666666666667

# Cleaning dataset

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...


In [ ]:
remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']

# Impute null values with None
def null_process(feature_df):
    col = 'src_text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return  text

In [ ]:
df = clean_dataset(df)
df["src_text"] = df.src_text.apply(nltk_preprocess)
df.head()

,true,src_text
0,unverified,vladimir putin netralized internal coup maybe ...
1,unverified,coup rt jimgeraghty rumor russian military att...
2,unverified,hoppla l0gg0l swiss rumor putin absence due gi...
3,unverified,putin reappears tv amid claim unwell threat co...
4,1,france 10 people dead shooting hq satirical we...


In [ ]:
def remove_date(row):
    return row['Created at'].timetz()

tweets_df['Created at'] = tweets_df.apply(lambda row: remove_date(row), axis=1)

tweets_df.head()

,ID,Text,Created at,User location,Num Followers,Num Friends,Num Favourites,User description,User verified,Language
0,937349434668498944,BREAKING: First NFL Team Declares Bankruptcy O...,15:54:54+00:00,"Sugar Land, TX",1712,2751,22141,Ofelia. Arizmendez @ deplorable me. I am a pro...,False,en
1,937379378006282240,BREAKING: First NFL Team Declares Bankruptcy O...,17:53:54+00:00,,14,99,125,,False,en
2,937380068590055425,BREAKING: First NFL Team Declares Bankruptcy O...,17:56:38+00:00,,14,99,125,,False,en
3,937429898670600192,BREAKING: First NFL Team Declares Bankruptcy O...,21:14:39+00:00,Ozarks. Missouri,1162,1258,9422,Happily married conservative Pentecostal woman...,False,en
4,937449906352152576,BREAKING: First NFL Team Declares Bankruptcy O...,22:34:09+00:00,,12772,13708,23094,Deplorable member of The Silenced Majority.. #...,False,en


In [ ]:
tweets_df2.to_csv("final_short.csv")

hello
